In [12]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 

In [13]:
#twitter api keys
auth = tweepy.OAuth1UserHandler(
   'ObDcu9DovwyYkx5w5tJouFGvP', '6A2UhUI9YpVis96xea1xmZqR5b2JIMIpUAOQIjJBaNPkINwnOj', '459226572-g5WB6p6QAcVciJ50uN6SLTRVfZKDqS0Bcp4bFGqd', 'jzFTq24nII5LUsG0FjsSCsrS0GLWN5ZK5BPIMfiJceUls')

api = tweepy.API(auth, wait_on_rate_limit=True)


In [14]:
search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
search_query = search_words + " -filter:retweets AND -filter:replies"

In [15]:
limit = 100000
tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

In [16]:
data=[]
for tweet in tweets:
    data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

Rate limit reached. Sleeping for: 750
Rate limit reached. Sleeping for: 759
Rate limit reached. Sleeping for: 754
Rate limit reached. Sleeping for: 757
Rate limit reached. Sleeping for: 765
Rate limit reached. Sleeping for: 772
Rate limit reached. Sleeping for: 758
Rate limit reached. Sleeping for: 763
Rate limit reached. Sleeping for: 763
Rate limit reached. Sleeping for: 762
Rate limit reached. Sleeping for: 747
Rate limit reached. Sleeping for: 751


In [17]:
columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
bp_tweets=pd.DataFrame(data, columns=columns)
bp_tweets.to_csv("BlackPanther-24-11.csv")

In [20]:
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
29545,29545,1592970443518251008,bergslyne,2022-11-16 19:58:44+00:00,"If there was a crossover, would Aquaman try to...",Wonderland,0,0,333,655,False,Twitter for iPhone
21370,21370,1593554279838937088,Jackjho40280159,2022-11-18 10:38:41+00:00,Quynh Alee viral video link: https://t.co/MHme...,NaN,0,3,0,10,False,Twitter Web App
24204,24204,1593310135732080641,breeeezygyal,2022-11-17 18:28:33+00:00,"Everyone, mostly females is in love with Shuri...",NaN,0,0,82,93,False,Twitter for iPhone
18581,18581,1593781616882880512,maryaofelia,2022-11-19 01:42:03+00:00,Vengeance has consumed us. We cannot let it co...,"Virac, Catanduanes Philippines",0,0,12,175,False,Twitter for iPhone
3619,3619,1595184630012985353,DieRoteLinie,2022-11-22 22:37:07+00:00,Schaut lieber #BlackAdam 👍👍👍\n\nDer Film war K...,Terra,0,1,2840,4994,False,Twitter for Android
34746,34746,1592687959001956352,bocha_kin,2022-11-16 01:16:14+00:00,昨日レイトショーで見て来たよ！！\n良かったー\nちょっとCGが雑だと感じた所もあったけどス...,NaN,0,2,2,5,False,Twitter for iPhone
26608,26608,1593121757036253189,lacosacine,2022-11-17 06:00:00+00:00,🎬 ¡Por si te lo perdiste!\n\n¿Cómo se eligió a...,Argentina,0,4,386142,546,True,Twitter Web App
23724,23724,1593340563528482846,AnJL_77,2022-11-17 20:29:27+00:00,#WakandaForever was alright. 😒 #long,🇺🇸,0,0,29,59,False,Twitter for Android
23947,23947,1593326556725841921,DeracheLyroy,2022-11-17 19:33:48+00:00,La découverte de Talokan avec Shuri et Namor a...,France,0,0,711,1702,False,Twitter for Android
402,402,1595651197398138880,Mika78107703,2022-11-24 05:31:05+00:00,I just want to thank Ms. Angela Bassett for pl...,"New York, NY",0,0,213,646,False,Twitter for Android


In [21]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35054 entries, 0 to 35053
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     35054 non-null  int64 
 1   id             35054 non-null  int64 
 2   username       35054 non-null  object
 3   time_of_tweet  35054 non-null  object
 4   tweet          35054 non-null  object
 5   location       24584 non-null  object
 6   retweets       35054 non-null  int64 
 7   likes          35054 non-null  int64 
 8   followers      35054 non-null  int64 
 9   following      35054 non-null  int64 
 10  verified       35054 non-null  bool  
 11  tweet_source   35054 non-null  object
dtypes: bool(1), int64(6), object(5)
memory usage: 3.0+ MB


In [48]:
cv = tweets['location'].replace(np.nan, "None")

In [56]:
cv[cv.str.contains('IL')][:60]


105               Crystal Lake, IL
409                    Chicago, IL
573                    Chicago, IL
720                    Chicago, IL
734          Elk Grove Village, IL
880     Chicago, IL and NW Suburbs
1142                   Chicago, IL
1256                   Chicago, IL
1526                  Chicago, IL.
1639                   Chicago, IL
1647                    BRAZILHOPE
1909          Obacht: SPOILERZONE!
2073                   Chicago, IL
2080                   Chicago, IL
2117                    Urbana, IL
2174                   Chicago, IL
2533          Obacht: SPOILERZONE!
2599                   Chicago, IL
2900                   Chicago, IL
3055                   Chicago, IL
3312                 CHICAGO,   IL
3793                   Chicago, IL
4021     Bottom of the map...FILA.
4073        CEBU CITY, PHILIPPINES
4188                   Chicago, IL
4335                   Chicago, IL
4475                   Chicago, IL
4608                   Chicago, IL
4766                